<a href="https://colab.research.google.com/github/AndrewEPrince/codespaces-blank/blob/main/models/ballsnet/BallsNetUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Installations
!pip install keras --upgrade -q

In [ ]:
#@title Imports
from google.colab import data_table
data_table.enable_dataframe_formatter()

import requests
import json
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt

from keras.models import load_model
from keras import backend as K

from joblib import load

!git clone https://github.com/AndrewEPrince/codespaces-blank.git

Cloning into 'codespaces-blank'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 123 (delta 69), reused 15 (delta 11), pack-reused 0
Receiving objects: 100% (123/123), 5.12 MiB | 6.02 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [4]:
#@title Load Model
K.clear_session()

model = load_model("/content/codespaces-blank/models/ballsnet/ballsnet1.2.keras")
scaler = load(open("/content/codespaces-blank/models/ballsnet/ballsnet_scaler.joblib", "rb"))

In [ ]:
#@title Get Data From API
#@markdown Enter a URL for a group
url = "https://ballchasing.com/api/groups/testing-rkmp6qipbj/" #@param {type: "string"}
#@markdown Enter your API key
api_key = '8wjl1G9d8V8Ymfju7hH8K57aKNUiNhYe5oOYOxNr' #@param {type: "string"}
headers = {'Authorization': api_key }
response = requests.get(url, headers=headers)
data = json.loads(response.text)
df = pd.json_normalize(data)



In [ ]:
#@title Prediciton Options
all_players = False #@param {type: "boolean"}
#@markdown If ```all_players = False```, enter the index of desired players e.g. 0,3,5,9 or 5 otherwise leave blank.

indices_of_players = '207,3,5' #@param {type: "string"}

In [ ]:
#@title Parse Data
#@markdown Only shows first 20 columns.
data_table.enable_dataframe_formatter()

for i in range(len(df)):
  data_df = pd.DataFrame(pd.json_normalize(df.iloc[i]['players']))
  for j in range(len(data_df)):
    temp_df = pd.DataFrame(data_df.iloc[j]).T
    player_df = temp_df if i == 0 and j == 0 else pd.concat([player_df, temp_df])

display(player_df.iloc[:, : 20])

temp_player_df = player_df.drop(columns=['platform'], axis=1)
temp_player_df = temp_player_df.drop(columns=['id'], axis=1)
names = temp_player_df['name']
temp_player_df = temp_player_df.drop(columns=['name'], axis=1)
temp_player_df = temp_player_df.drop(columns=['team'], axis=1)
temp_player_df = temp_player_df.drop(columns=['cumulative.games'], axis=1)

x = np.asarray(temp_player_df)

x_scaled = scaler.transform(x)

In [ ]:
#@title Run Prediction
# Convert indices_of_players to a list of integers
selected_indices = [int(i) for i in indices_of_players.split(',')]

if all_players:
    # Use all players
    x_scaled_selected = x_scaled
    selected_names = names
else:
    # Use only the selected players if the indices are within the valid range
    if all(index < len(x_scaled) for index in selected_indices):
        x_scaled_selected = x_scaled[selected_indices]
        selected_names = [names[i] for i in selected_indices]
    else:
        print("Invalid indices provided. Using all players.")
        x_scaled_selected = x_scaled
        selected_names = names

predictions = model.predict(x_scaled_selected, batch_size=None, verbose=1)

classes = [
    "Premier",
    "Master",
    "Elite",
    "Veteran",
    "Rival",
    "Challenger",
    "Prospect",
    "Contender",
    "Amateur"
]

num_classes = len(classes)

for i, (pred, player_name) in enumerate(zip(predictions, selected_names)):
    plt.figure(figsize=(6, 4))
    plt.bar(range(num_classes), pred)
    plt.xlabel('Rank')
    plt.ylabel('Probability')
    plt.title(f'{player_name}')
    plt.xticks(range(num_classes), classes, rotation=45, ha='right')
    plt.tight_layout()
    plt.show()